In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("udf").getOrCreate()

22/07/30 23:19:22 WARN Utils: Your hostname, singyeongdeog-ui-Macmini.local resolves to a loopback address: 127.0.0.1; using 222.98.22.103 instead (on interface en0)
22/07/30 23:19:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/30 23:19:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]

In [4]:
df = spark.createDataFrame(data=transactions, schema=schema)

In [5]:
df.createOrReplaceTempView("transactions")

In [6]:
spark.sql("SELECT * FROM transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [8]:
def squared(n):
    return n*n

spark.udf.register("squared", squared)

<function __main__.squared(n)>

In [9]:
spark.sql("SELECT price, squared(price) FROM transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [10]:
spark.sql("SELECT price, squared(price) FROM transactions").printSchema()

root
 |-- price: long (nullable = true)
 |-- squared(price): string (nullable = true)



In [11]:
from pyspark.sql.types import LongType

def squared(n):
    return n*n

spark.udf.register("squared", squared, LongType())

22/07/30 23:25:31 WARN SimpleFunctionRegistry: The function squared replaced a previously registered function.


<function __main__.squared(n)>

In [12]:
spark.sql("SELECT price, squared(price) FROM transactions").printSchema()

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [14]:
from pyspark.sql.types import LongType
from pyspark.sql.functions import udf

@udf("long")
def squared(n):
    return n*n
spark.sql("SELECT price, squared(price) FROM transactions").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|22000|     484000000|
|21500|     462250000|
|42000|    1764000000|
|21250|     451562500|
|68700|    4719690000|
|32000|    1024000000|
+-----+--------------+



In [16]:
def read_number(n):
    units = ["","십","백","천","만"]
    nums = '일이삼사오육칠팔구'
    result = []
    i = 0
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1] + units[i])
        i += 1
    return "".join(reversed(result))
    

print(read_number(21250))
print(read_number(68700))

이만일천이백오십
육만팔천칠백


In [17]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [18]:
spark.sql("SELECT price, read_number(price) FROM transactions").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [19]:
def get_weekday(date):
    import calendar
    return calendar.day_name[date.weekday()]

spark.udf.register("get_weekday", get_weekday)

<function __main__.get_weekday(date)>

In [20]:
query = """
SELECT
    datetime,
    get_weekday(TO_DATE(datetime)) as day_of_week
FROM
    transactions
"""
spark.sql(query).show()

+-------------------+-----------+
|           datetime|day_of_week|
+-------------------+-----------+
|2021-11-07 13:20:00|     Sunday|
|2021-10-24 11:19:00|     Sunday|
|2021-07-25 11:12:40|     Sunday|
|2021-07-10 08:20:00|   Saturday|
|2021-07-01 05:36:00|   Thursday|
|2020-08-19 19:04:00|  Wednesday|
+-------------------+-----------+

